# Spatial predictors

A key aspect of focal site multi-scale studies is the selection of spatial predictors, which are ultimately used as explanatory variables to study the effect of spatial pattern on a response variable. In landscape-species relationships, landscape patterns are often computed using

In [ ]:
import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
import seaborn as sns

import focalpy

In [ ]:
stations_filepath = "data/stations.gpkg"

buildings_filepath = "data/buildings.gpkg"
dem_filepath = "data/dem.tif"
lulc_filepath = "data/lulc.tif"

buffer_dists = [50, 100, 250, 500]

In [ ]:
station_gser = gpd.read_file(stations_filepath).set_index("station_id")["geometry"]

In [ ]:
fig, ax = plt.subplots()
for buffer_dist, color in zip(buffer_dists, sns.color_palette()):
    station_gser.buffer(buffer_dist).plot(
        facecolor=(0, 0, 0, 0), edgecolor=color, ax=ax
    )
cx.add_basemap(ax, crs=station_gser.crs)

In [ ]:
# buildings
building_gdf = gpd.read_file(buildings_filepath).set_index("id")
building_gdf.head()

In [ ]:
# dem
with rio.open(dem_filepath) as src:
    # read the one and only band
    dem_arr = src.read(1)
    dem_transform = src.transform
    dem_nodata = src.nodata

## Computing features
### Vector data
#### Basic geopandas operations

In [ ]:
focalpy.compute_vector_features(
    building_gdf, station_gser, buffer_dists, gb_reduce_method="size"
)

In [ ]:
focalpy.compute_vector_features(building_gdf, station_gser, buffer_dists, "mean")

In [ ]:
focalpy.compute_vector_features(
    building_gdf, station_gser, buffer_dists, {"area": "sum", "height": "mean"}
)

In [ ]:
focalpy.compute_vector_features(
    building_gdf, station_gser, buffer_dists, ["sum", "mean"]
)

In [ ]:
focalpy.compute_vector_features(
    building_gdf.assign(**{"volume": building_gdf["area"] * building_gdf["height"]}),
    station_gser,
    buffer_dists,
    {"volume": "mean"},
)

#### Urban morphometrics (momepy)

In [ ]:
focalpy.compute_urban_morphometrics(
    building_gdf,
    station_gser,
    buffer_dists,
    ["circular_compactness", "fractal_dimension"],
    "mean",
)

In [ ]:
focalpy.compute_urban_morphometrics(
    building_gdf,
    station_gser,
    buffer_dists,
    ["circular_compactness", "form_factor"],
    "mean",
    momepy_metrics_kwargs_dict={"form_factor": {"height": building_gdf["height"]}},
)

### Raster data
#### Basic raster operations (zonal statistics with rasterstats)

In [ ]:
focalpy.compute_raster_features(
    dem_arr,
    station_gser,
    buffer_dists,
    affine=dem_transform,
    stats="mean",
    nodata=dem_nodata,
)

In [ ]:
with rio.open(lulc_filepath) as src:
    # read the one and only band
    lulc_arr = src.read(1)
    lulc_transform = src.transform

target_class = 5  # urban green spaces
focalpy.compute_raster_features(
    lulc_arr == target_class,
    station_gser,
    buffer_dists,
    affine=lulc_transform,
    stats=["sum", "mean"],
)

### Landscape metrics (pylandstats)

In [ ]:
# green spaces (5), meadows/pastures (8) and forests (10)
focalpy.compute_landscape_metrics(
    lulc_filepath,
    station_gser,
    buffer_dists,
    class_metrics=["proportion_of_landscape", "edge_density"],
    landscape_metrics=["shannon_diversity_index"],
    classes=[5, 8, 10],
)

### Terrain attributes (xDEM)

In [ ]:
focalpy.compute_terrain_attributes(
    dem_filepath, station_gser, buffer_dists, "slope", stats=["mean", "max"]
)

In [ ]:
focalpy.compute_terrain_attributes(
    dem_filepath,
    station_gser,
    buffer_dists,
    ["slope", "topographic_position_index"],
    stats="mean",
)

In [ ]:
focalpy.compute_terrain_attributes(
    dem_filepath,
    station_gser,
    buffer_dists,
    ["slope", "topographic_position_index"],
    stats=["mean", "max"],
)


###

In [ ]:
fc = focalpy.core.FeatureComputer(station_gser)
fc.compute_features_df(
    "compute_landscape_metrics",
    lulc_filepath,
    buffer_dists,
    feature_methods_kwargs_dict=dict(
        class_metrics=["proportion_of_landscape", "edge_density"],
        landscape_metrics=["shannon_diversity_index"],
        classes=[5, 8, 10],
    ),
)